<a href="https://colab.research.google.com/github/lohansilva/analise-covid19/blob/main/Coleta_automatica_de_dados_covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Autor: Lohan Italo de Souza Almeida Silva
### Data: 26/04/2023
**Bibliotecas utilizadas:**
- requests
- pandas
- xlsxwriter

**Fonte dos dados:**
- https://ourworldindata.org/covid-cases

**Resumo:**
Criei uma análise rápida sobre os dados obtidos no site acima que consiste em filtrar os dados por localização, para que pudesse analisar somente o Brasil(mas pode ser alterado no código), e apliquei um filtro de período(pode ser alterado também) para que tivesse uma visualização melhor do gráfico para verificar a quantidade de novas mortes por dia e quantidade de novos casos por dia no Brasil.

**Desafios durante :**
- gerar e exportar gráficos para o excel
- aprender a utilizar a biblioteca xlswriter

**Outra alternativa usando selenium para fazer o download do arquivo em csv:**
- https://github.com/lohansilva/analise-covid19

**Alguns links que utilizei para me auxiliar no projeto:**
- https://stackoverflow.com/questions/65278555/typeerror-nan-inf-not-supported-in-write-number-without-nan-inf-to-errors-w
- https://www.youtube.com/watch?v=D13sHx-7n0U
- https://www.youtube.com/watch?v=eHZ_d0bINkM
- https://www.youtube.com/watch?v=8AMNaVt0z_M&t=899s
- https://xlsxwriter.readthedocs.io/getting_started.html
- https://stackoverflow.com/questions/72754651/attributeerror-webdriver-object-has-no-attribute-find-element-by-xpath
- https://dataguru.cc/step-by-step-guide-to-install-selenium-in-google-colab/

In [ ]:
# instalação do XlsxWriter para salvar os graficos no excel
!pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 5.6 MB/s eta 0:00:00


In [ ]:
#importando as bibliotecas necessárias para leitura dos dados, filtragem e exportação dos dados em excel
import requests
import pandas as pd
import xlsxwriter

In [ ]:
# Faz a requisição para a página
url = 'https://covid.ourworldindata.org/data/owid-covid-data.csv'
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36"}
r = requests.get(url,headers=headers)

In [ ]:
# Cria um DataFrame a partir dos dados extraídos
covid_data = pd.read_csv(url)

In [ ]:
# Filtrar dados do Brasil e em um determinado periodo
brasil_data = covid_data[(covid_data['location'] == 'Brazil') & (covid_data['date'] >= '2023-01-01') & (covid_data['date'] <= '2023-01-05')]

In [ ]:
# limpar colunas, para deixar apenas as importantes para análise
brasil_data = brasil_data[['date', 'new_cases', 'new_deaths']]

In [ ]:
# Criar novo arquivo Excel
workbook = xlsxwriter.Workbook('analise_covid.xlsx')

In [ ]:
# se tiver dados nan, alterar para o valor "0", para evitar erros
brasil_data = brasil_data.fillna(0)

In [ ]:
#criar planilha com os dados
worksheet = workbook.add_worksheet('Dados')
worksheet.write_row(0, 0, brasil_data.columns)
for i, row in enumerate(brasil_data.values):
    worksheet.write_row(i+1, 0, row)

In [ ]:
# Adicionar gráfico de linhas sobre as novas mortes por covid 
chart = workbook.add_chart({'type': 'line'})
chart.add_series({
    'categories': ['Dados', 1, 0, len(brasil_data), 0],
    'values': ['Dados', 1, 2, len(brasil_data), 2],
    'name': 'Mortes Confirmadas'
})

chart.set_title({'name': 'Evolução de mortes por COVID-19 no Brasil(5 dias aleatorios)'})
chart.set_x_axis({'name': 'Data'})
chart.set_y_axis({'name': 'Número de novas mortes'})
worksheet.insert_chart('F2', chart)


0

In [ ]:
# Adicionar gráfico de linhas sobre os novos casos confirmados
chart2 = workbook.add_chart({'type': 'line'})
chart2.set_y_axis({'name': 'Casos confirmados','min': 0, 'max': 35000})
chart2.add_series({
    'categories': ['Dados', 1, 0, len(brasil_data), 0],
    'values': ['Dados', 1, 1, len(brasil_data), 1],
    'name': 'Casos confirmados'
})
chart2.set_title({'name': 'Evolução de novos casos confirmados por COVID-19 no Brasil(5 dias aleatorios)'})
chart2.set_x_axis({'name': 'Data'})
chart2.set_y_axis({'name': 'Número de casos'})
worksheet.insert_chart('N2', chart2)

0

In [ ]:
# Salvar arquivo
workbook.close()